In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as spa
import time

import sys; sys.path.insert(0, '..')

import hylife.utilitis_FEEC.basics.spline_evaluation_3d as eva_3d

import hylife.utilitis_FEEC.bsplines     as bsp 
import hylife.utilitis_FEEC.Bspline      as bspline
import hylife.utilitis_FEEC.spline_space as spl

import hylife.utilitis_PIC.STRUPHY_fields     as pic1
import hylife.utilitis_PIC.STRUPHY_fields_new as pic1_new

import hylife.utilitis_PIC.STRUPHY_accumulation_kernels as pic2
import hylife.utilitis_PIC.accumulation_kernels         as pic2_new

In [2]:
# setup
Nel        = [8, 6, 10]                    # number of elements on logical domain
bc         = [False, True, False]          # boundary conditions (True: periodic, False: else)
p          = [3, 2, 2]                     # spline degrees  

el_b       = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                       # element boundaries
T          = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]   # knot vectors (for N functions)

nq_el      = [6, 6, 6]                     # number of quadrature points per element for integrations over whole domain

spaces     = [spl.spline_space_1d(T, p, bc, nq_el) for T, p, bc, nq_el in zip(T, p, bc, nq_el)] # 1d B-spline spaces
t_space    = spl.tensor_spline_space(spaces)

N          = [bspline.Bspline(spl.T, spl.p)     for spl in spaces]     # N-splines for comparison
D          = [bspline.Bspline(spl.t, spl.p - 1) for spl in spaces]     # D-splines for comparison

kind_map   = 1
params_map = [1., 2., 3.]

Np = int(1e6) # number of particles

# spline coefficients
u1 = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2])
u2 = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2])
u3 = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2])

b1 = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseD[2])
b2 = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseD[2])
b3 = np.random.rand(t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseN[2])

# particles
particles_pos = np.random.rand(Np, 3)

particles     = np.random.rand(Np, 7)

u_part     = np.empty((Np, 3), dtype=float, order='F')
u_part_new = np.empty((Np, 3), dtype=float)

b_part     = np.empty((Np, 3), dtype=float, order='F')
b_part_new = np.empty((Np, 3), dtype=float)

mat12 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat13 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat23 = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')

mat12_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat13_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat23_new = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)

In [3]:
# ==== coefficients for pp-forms in interval [0, delta] (N and D) for fast field evaluation
pp0 = []
pp1 = []

delta = t_space.delta

for i in range(3):
    
    if   p[i] == 1:
        pp0.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]]))
        pp1.append(np.asfortranarray([[1/delta[i]]]))
    
    elif p[i] == 2:
        pp0.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]]))
        pp1.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]])/delta[i])
    
    elif p[i] == 3:
        pp0.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]]))
        pp1.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]])/delta[i])
    
    elif p[i] == 4:
        pp0.append(np.asfortranarray([[1/24, -1/(6*delta[i]), 1/(4*delta[i]**2), -1/(6*delta[i]**3), 1/(24*delta[i]**4)], [11/24, -1/(2*delta[i]), -1/(4*delta[i]**2), 1/(2*delta[i]**3), -1/(6*delta[i]**4)], [11/24, 1/(2*delta[i]), -1/(4*delta[i]**2), -1/(2*delta[i]**3), 1/(4*delta[i]**4)], [1/24, 1/(6*delta[i]), 1/(4*delta[i]**2), 1/(6*delta[i]**3), -1/(6*delta[i]**4)], [0., 0., 0., 0., 1/(24*delta[i]**4)]]))
        pp1.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]])/delta[i])
    
    else:
        print('Spline degree > 4 not yet implemented!')
# ======================================================================


# number of basis functions in different spaces
NbaseN = t_space.NbaseN
NbaseD = t_space.NbaseD

Nbase_0form =  [NbaseN[0], NbaseN[1], NbaseN[2]]
Nbase_1form = [[NbaseD[0], NbaseN[1], NbaseN[2]], [NbaseN[0], NbaseD[1], NbaseN[2]], [NbaseN[0], NbaseN[1], NbaseD[2]]]
Nbase_2form = [[NbaseN[0], NbaseD[1], NbaseD[2]], [NbaseD[0], NbaseN[1], NbaseD[2]], [NbaseD[0], NbaseD[1], NbaseN[2]]]
Nbase_3form =  [NbaseD[0], NbaseD[1], NbaseD[2]]

Ntot_0form  =  NbaseN[0]*NbaseN[1]*NbaseN[2] 
Ntot_1form  = [NbaseD[0]*NbaseN[1]*NbaseN[2], NbaseN[0]*NbaseD[1]*NbaseN[2], NbaseN[0]*NbaseN[1]*NbaseD[2]]
Ntot_2form  = [NbaseN[0]*NbaseN[1]*NbaseD[2], NbaseD[0]*NbaseN[1]*NbaseD[2], NbaseD[0]*NbaseD[1]*NbaseN[2]]  
Ntot_3form  =  NbaseD[0]*NbaseD[1]*NbaseD[2]

In [4]:
# evaluation
timea = time.time()
pic1.evaluate_1form(np.asfortranarray(particles_pos), T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], p, Nel, np.asfortranarray(Nbase_1form), Np, np.asfortranarray(u1), np.asfortranarray(u2), np.asfortranarray(u3), pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], u_part, kind_map, params_map)
timeb = time.time()

In [5]:
timeb-timea

1.0221996307373047

In [6]:
timea = time.time()
pic1_new.evaluate_1form(particles_pos, T[0], T[1], T[2], p, Nel, t_space.NbaseN, t_space.NbaseD, Np, u1, u2, u3, u_part_new, kind_map, params_map)
timeb = time.time()

In [7]:
timeb-timea

0.2301483154296875

In [8]:
np.allclose(u_part, u_part_new)

True

In [9]:
# evaluation
timea = time.time()
pic1.evaluate_2form(np.asfortranarray(particles_pos), T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], p, Nel, np.asfortranarray(Nbase_2form), Np, np.asfortranarray(b1), np.asfortranarray(b2), np.asfortranarray(b3), pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], b_part, kind_map, params_map)
timeb = time.time()

In [10]:
timeb-timea

0.6132717132568359

In [11]:
timea = time.time()
pic1_new.evaluate_2form(particles_pos, T[0], T[1], T[2], p, Nel, t_space.NbaseN, t_space.NbaseD, Np, b1, b2, b3, b_part_new, kind_map, params_map)
timeb = time.time()

In [12]:
timeb-timea

0.21719741821289062

In [13]:
np.allclose(b_part, b_part_new)

True

In [14]:
timea = time.time()
pic2.kernel_step1(np.asfortranarray(particles), p, Nel, [t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseD[2]], T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], b_part, kind_map, params_map, mat12, mat13, mat23)
timeb = time.time()

In [15]:
timeb - timea

4.070690631866455

In [16]:
timea = time.time()
pic2_new.kernel_step1(particles, T[0], T[1], T[2], p, Nel, t_space.NbaseN, t_space.NbaseD, Np, b_part_new, kind_map, params_map, mat12_new, mat13_new, mat23_new)
timeb = time.time()

In [17]:
timeb - timea

2.692026376724243

In [18]:
print(np.allclose(mat12, mat12_new))
print(np.allclose(mat13, mat13_new))
print(np.allclose(mat23, mat23_new))

True
True
True


In [19]:
mat11 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat12 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat13 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat22 = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat23 = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')
mat33 = np.empty((t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float, order='F')

vec1 = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2]), dtype=float, order='F')
vec2 = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2]), dtype=float, order='F')
vec3 = np.empty((t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2]), dtype=float, order='F')


mat11_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat12_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat13_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat22_new = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat23_new = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)
mat33_new = np.empty((t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2], 2*p[0] + 1, 2*p[1] + 1, 2*p[2] + 1), dtype=float)

vec1_new = np.empty((t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2]), dtype=float)
vec2_new = np.empty((t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2]), dtype=float)
vec3_new = np.empty((t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2]), dtype=float)

In [20]:
timea = time.time()
pic2.kernel_step3(np.asfortranarray(particles), p, Nel, [t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseD[2]], T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], b_part, kind_map, params_map, mat11, mat12, mat13, mat22, mat23, mat33, vec1, vec2, vec3)
timeb = time.time()

In [21]:
timeb - timea

10.540645122528076

In [22]:
timea = time.time()
pic2_new.kernel_step3(particles, T[0], T[1], T[2], p, Nel, t_space.NbaseN, t_space.NbaseD, Np, b_part_new, kind_map, params_map, mat11_new, mat12_new, mat13_new, mat22_new, mat23_new, mat33_new, vec1_new, vec2_new, vec3_new)
timeb = time.time()

In [23]:
timeb - timea

6.859557151794434

In [24]:
print(np.allclose(vec1, vec1_new))
print(np.allclose(vec2, vec2_new))
print(np.allclose(vec3, vec3_new))

print(np.allclose(mat11, mat11_new))
print(np.allclose(mat12, mat12_new))
print(np.allclose(mat13, mat13_new))
print(np.allclose(mat22, mat22_new))
print(np.allclose(mat23, mat23_new))
print(np.allclose(mat33, mat33_new))

True
True
True
True
True
True
True
True
True
